### Rilevazioni mensili - beni e servizi di largo consumo
Web Scraping

# !! 
Eseguire questo codice assicurandosi che il computer non entri mai in modalità standby.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

In [3]:
# URL della pagina
url = 'https://osservaprezzi.mise.gov.it/prezzi/livelli/beni-e-servizi-di-largo-consumo/archivio-rilevazioni-beni-e-servizi-di-largo-consumo'

# Making a GET request
r = requests.get(url)

# check status code for response received
# success code - 200
print(r)

# print content of request
print(r.content)

<Response [200]>
b'<!DOCTYPE html>\n<!--[if IE 8]><html class="no-js ie89 ie8" lang="it-it"><![endif]-->\n<!--[if IE 9]><html class="no-js ie89 ie9" lang="it-it"><![endif]-->\n<!--[if (gte IE 9)|!(IE)]><!-->\n<html class="no-js theme-italia" lang="it-it" dir="ltr">\n<!--<![endif]-->\n<head>\n\t<meta charset="utf-8" />\n\t<meta http-equiv="x-ua-compatible" content="ie=edge">\n\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\n\t\t<!--[if lt IE 9]><script src="/media/jui/js/html5.js"></script><![endif]-->\n\t<!-- include html5shim per Explorer 8 -->\n\t<script src="https://osservaprezzi.mise.gov.it/templates/italiapa/build/vendor/modernizr.js"></script>\n\n\t<script>__PUBLIC_PATH__ = \'https://osservaprezzi.mise.gov.it/templates/italiapa/build/\'</script>\n\t<script>__DEFAULT_THEME__ = \'italia\'</script>\n\n\t<link href=\'//fonts.googleapis.com/css?family=Titillium+Web:400,400italic,700,\' rel=\'stylesheet\' type=\'text/css\' />\n\t<link href="https://fonts.google

In [4]:
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 8]><html class="no-js ie89 ie8" lang="it-it"><![endif]-->
<!--[if IE 9]><html class="no-js ie89 ie9" lang="it-it"><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!-->
<html class="no-js theme-italia" dir="ltr" lang="it-it">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!--[if lt IE 9]><script src="/media/jui/js/html5.js"></script><![endif]-->
  <!-- include html5shim per Explorer 8 -->
  <script src="https://osservaprezzi.mise.gov.it/templates/italiapa/build/vendor/modernizr.js">
  </script>
  <script>
   __PUBLIC_PATH__ = 'https://osservaprezzi.mise.gov.it/templates/italiapa/build/'
  </script>
  <script>
   __DEFAULT_THEME__ = 'italia'
  </script>
  <link href="//fonts.googleapis.com/css?family=Titillium+Web:400,400italic,700," rel="stylesheet" type="text/css"/>
  <link href="https://fonts.googleapis.com/icon?family=Material+Icon

In [5]:
# Esempio per dicembre 2022
params = {
    'ANNO': '2022',
    'MESE': '12',
    'f[provincia]': 'Bolzano',
    'f[tipo_record_mise]': 'energia',
    'submit': 'Applica'
}

# Effettuare la richiesta GET con i parametri
response = requests.get(url, params=params)
response.raise_for_status()  # verifica che la richiesta sia andata a buon fine

# Parsing dell'HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Trovare la tabella specifica contenente i dati
table = soup.find('table')

# Estrarre i dati dalla tabella
data_bolzano_2022 = []

# Iterare attraverso le righe della tabella
for row in table.find_all('tr')[1:]:  # Ignorare l'intestazione
    cols = row.find_all('td')
    data_bolzano_2022.append([col.text.strip() for col in cols])

# Stampare i dati estratti
for data in data_bolzano_2022:
    print(data)

['Gasolio Per Auto Con Servizio Alla Pompa (1 L)', '1.79', '2.364', '1.948', 'Energetici']
['Gasolio Per Auto Senza Servizio Alla Pompa (1 L)', '1.699', '2.064', '1.851', 'Energetici']
['Benzina Verde Senza Servizio Alla Pompa (1 L)', '1.599', '1.964', '1.77', 'Energetici']
['Benzina Verde Con Servizio Alla Pompa (1 L)', '1.769', '2.264', '1.885', 'Energetici']
['Gas GPL (1 L)', '0.759', '0.899', '0.837', 'Energetici']
['Gas Metano Per Autotrazione (1 Kg)', '1.399', '3.799', '2.43', 'Energetici']


# !! 
Modificare i parametri sottostanti per scaricare i dati desiderati.

Nota: Questo blocco di codice scarica esclusivamente i dati per la provincia di Bolzano. Per scaricare dati relativi ad altre province italiane, consultare la sezione successiva.

In [6]:
# Parametri da utilizzare
ANNO = list(range(2021, 2025))
# list(range(annoinizio, annofine+1))
# list(range(2021, 2025)) per serie di più anni - in questo esempio dal 2021 (primo valore incluso) al 2024 (secondo valore escluso)
# [2024] per anno singolo

MESE = list(range(1, 13))
# list(range(meseinizio, mesefine+1))
# list(range(1, 13)) per anno completo, da gennaio a dicembre
# [7] per mese singolo

TIPO_RECORD_MISE = ["altri_alim", "energia", "groc", "ittici", "orto", "servizi"]

print(ANNO)
print(MESE)
print(TIPO_RECORD_MISE)

[2021, 2022, 2023, 2024]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
['altri_alim', 'energia', 'groc', 'ittici', 'orto', 'servizi']


In [7]:
# Lista per memorizzare tutti i dati
all_data = []

# Funzione per fare una richiesta e parsare i dati
def fetch_data(anno, mese, tipo_record):
    params = {
        'ANNO': anno,
        'MESE': mese,
        'f[provincia]': 'Bolzano',
        'f[tipo_record_mise]': tipo_record,
        'submit': 'Applica'
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # verifica che la richiesta sia andata a buon fine
    
    # Parsing dell'HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Trovare la tabella specifica contenente i dati
    table = soup.find('table')
    
    if table:
        # Estrarre i dati dalla tabella
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            row_data = [col.text.strip() for col in cols]
            row_data.insert(0, anno)
            row_data.insert(1, mese)
            all_data.append(row_data)
    else:
        print(f"Tabella non trovata per ANNO={anno}, MESE={mese}, TIPO_RECORD={tipo_record}")

# Parallelizzazione delle richieste
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for anno in ANNO:
        for mese in MESE:
            for tipo_record in TIPO_RECORD_MISE:
                futures.append(executor.submit(fetch_data, anno, mese, tipo_record))
    
    for future in concurrent.futures.as_completed(futures):
        future.result()  # Verifica se ci sono eccezioni

# Creazione del DataFrame
columns = ['Anno', 'Mese', 'DescrizioneProdotto', 'QuotazioneMinima', 'QuotazioneMassima', 'QuotazioneMedia', 'TipologiaProdotto']
df_BZ = pd.DataFrame(all_data, columns=columns)
df_BZ

,Anno,Mese,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2021,3,Gasolio Per Auto Con Servizio Alla Pompa (1 Cl),1.383,1.954,1.563,Energetici
1,2021,3,Gasolio Per Auto Senza Servizio Alla Pompa (1 Cl),1.383,1.629,1.499,Energetici
2,2021,3,Benzina Verde Senza Servizio Alla Pompa (1 Cl),1.426,1.714,1.618,Energetici
3,2021,3,Benzina Verde Con Servizio Alla Pompa (1 Cl),1.531,1.984,1.685,Energetici
4,2021,3,Gas GPL (1 Cl),0.659,0.764,0.691,Energetici
...,...,...,...,...,...,...,...
5146,2024,7,Taglio Capelli Uomo (1 Pz),12,27,21.39,Servizi
5147,2024,7,Servizio Di Barba E Baffi (1 Pz),10,25,14.91,Servizi
5148,2024,7,Taglio Capelli Donna (1 Pz),15,35,22.02,Servizi
5149,2024,7,Messa In Piega (1 Pz),12,35,20.92,Servizi


In [8]:
df_BZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Anno                 5151 non-null   int64 
 1   Mese                 5151 non-null   int64 
 2   DescrizioneProdotto  5151 non-null   object
 3   QuotazioneMinima     5151 non-null   object
 4   QuotazioneMassima    5151 non-null   object
 5   QuotazioneMedia      5151 non-null   object
 6   TipologiaProdotto    5151 non-null   object
dtypes: int64(2), object(5)
memory usage: 281.8+ KB


In [9]:
# DataFrame ordinato per 'Tipologia di Prodotto', 'Anno', 'Mese', e 'Descrizione Prodotto'
df_BZ = df_BZ.sort_values(['Anno', 'Mese', 'TipologiaProdotto', 'DescrizioneProdotto']).reset_index(drop=True)
df_BZ

,Anno,Mese,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2021,1,"Carne Fresca Bovino Adulto, Primo Taglio (1000...",12.79,33.5,19.67,Alimentari
1,2021,1,Carne Fresca Suina Con Osso (1000 Gr),4.49,12,8.27,Alimentari
2,2021,1,Pane Fresco (1000 Gr),3.99,6,5.37,Alimentari
3,2021,1,Parmigiano Reggiano (1000 Gr),15.99,27.6,21.64,Alimentari
4,2021,1,Petto Di Pollo (1000 Gr),7.5,15.95,10.75,Alimentari
...,...,...,...,...,...,...,...
5146,2024,7,Soprattacco Scarpe Donna (1 Pz),6,24,13.43,Servizi
5147,2024,7,Taglio Capelli Donna (1 Pz),15,35,22.02,Servizi
5148,2024,7,Taglio Capelli Uomo (1 Pz),12,27,21.39,Servizi
5149,2024,7,Toilette Cani (1 Pz),35,55,42.5,Servizi


In [10]:
df_BZ.to_csv(f'prezzi_BZ_{df_BZ["Anno"].iloc[0]}-{df_BZ["Mese"].iloc[0]:02d}_{df_BZ["Anno"].iloc[-1]}-{df_BZ["Mese"].iloc[-1]:02d}.csv', index=False)
df_BZ.to_excel(f'prezzi_BZ_{df_BZ["Anno"].iloc[0]}-{df_BZ["Mese"].iloc[0]:02d}_{df_BZ["Anno"].iloc[-1]}-{df_BZ["Mese"].iloc[-1]:02d}.xlsx', index=False)

In [11]:
data_BZ = pd.read_csv("prezzi_BZ_2021-01_2024-07.csv")
data_BZ

,Anno,Mese,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2021,1,"Carne Fresca Bovino Adulto, Primo Taglio (1000...",12.79,33.50,19.67,Alimentari
1,2021,1,Carne Fresca Suina Con Osso (1000 Gr),4.49,12.00,8.27,Alimentari
2,2021,1,Pane Fresco (1000 Gr),3.99,6.00,5.37,Alimentari
3,2021,1,Parmigiano Reggiano (1000 Gr),15.99,27.60,21.64,Alimentari
4,2021,1,Petto Di Pollo (1000 Gr),7.50,15.95,10.75,Alimentari
...,...,...,...,...,...,...,...
5146,2024,7,Soprattacco Scarpe Donna (1 Pz),6.00,24.00,13.43,Servizi
5147,2024,7,Taglio Capelli Donna (1 Pz),15.00,35.00,22.02,Servizi
5148,2024,7,Taglio Capelli Uomo (1 Pz),12.00,27.00,21.39,Servizi
5149,2024,7,Toilette Cani (1 Pz),35.00,55.00,42.50,Servizi


In [12]:
data_BZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Anno                 5151 non-null   int64  
 1   Mese                 5151 non-null   int64  
 2   DescrizioneProdotto  5151 non-null   object 
 3   QuotazioneMinima     5151 non-null   float64
 4   QuotazioneMassima    5151 non-null   float64
 5   QuotazioneMedia      5151 non-null   float64
 6   TipologiaProdotto    5151 non-null   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 281.8+ KB


### PROVINCE ITALIANE

In [13]:
# Funzione per estrarre i nomi delle province dal codice HTML
def get_province_list(url):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    province_select = soup.find('select', {'name': 'f[PROVINCIA]'})
    province = [option.text.strip() for option in province_select.find_all('option') if option.text.strip()]
    
    return province

# Estrai la lista delle province
PROVINCE = get_province_list(url)
PROVINCE

['Alessandria',
 'Ancona',
 'Aosta',
 'Arezzo',
 'Ascoli Piceno',
 'Avellino',
 'Bari',
 'Belluno',
 'Benevento',
 'Bergamo',
 'Biella',
 'Bologna',
 'Bolzano',
 'Brescia',
 'Cagliari',
 'Catanzaro',
 'Cosenza',
 'Cremona',
 'Cuneo',
 'Ferrara',
 'Firenze',
 'Forli',
 'Genova',
 'Gorizia',
 'Grosseto',
 'Lecco',
 'Livorno',
 'Lodi',
 'Lucca',
 'Macerata',
 'Mantova',
 'Messina',
 'Milano',
 'Modena',
 'Napoli',
 'Novara',
 'Padova',
 'Palermo',
 'Parma',
 'Perugia',
 'Pescara',
 'Piacenza',
 'Pistoia',
 'Pordenone',
 'Ravenna',
 'Reggio Calabria',
 'Reggio Emilia',
 'Rimini',
 'Roma',
 'Rovigo',
 'Sassari',
 'Siena',
 'Siracusa',
 'Terni',
 'Torino',
 'Trento',
 'Treviso',
 'Trieste',
 'Udine',
 'Varese',
 'Venezia',
 'Vercelli',
 'Verona',
 'Vicenza']

# !! 
Modificare i parametri sottostanti per scaricare i dati desiderati.

In [14]:
# Parametri da utilizzare
ANNO = [2024]
# list(range(annoinizio, annofine+1))
# list(range(2022, 2025)) per serie di più anni - in questo esempio dal 2021 (primo valore incluso) al 2024 (secondo valore escluso)
# [2024] per anno singolo

MESE = list(range(1, 8))
# list(range(meseinizio, mesefine+1))
# list(range(1, 13)) per anno completo, da gennaio a dicembre
# [7] per mese singolo

TIPO_RECORD_MISE = ["altri_alim", "energia", "groc", "ittici", "orto", "servizi"]

print(ANNO)
print(MESE)
print(TIPO_RECORD_MISE)

[2024]
[1, 2, 3, 4, 5, 6, 7]
['altri_alim', 'energia', 'groc', 'ittici', 'orto', 'servizi']


In [15]:
# Lista per memorizzare tutti i dati
all_data = []

# Funzione per fare una richiesta e parsare i dati
def fetch_data(anno, mese, tipo_record, provincia):
    params = {
        'ANNO': anno,
        'MESE': mese,
        'f[provincia]': provincia,
        'f[tipo_record_mise]': tipo_record,
        'submit': 'Applica'
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # verifica che la richiesta sia andata a buon fine
    
    # Parsing dell'HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Trovare la tabella specifica contenente i dati
    table = soup.find('table')
    
    if table:
        # Estrarre i dati dalla tabella
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            row_data = [col.text.strip() for col in cols]
            row_data.insert(0, anno)
            row_data.insert(1, mese)
            row_data.insert(2, provincia)
            all_data.append(row_data)
    else:
        print(f"Tabella non trovata per ANNO={anno}, MESE={mese}, PROVINCIA={provincia}, TIPO_RECORD={tipo_record}")

# Parallelizzazione delle richieste
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for provincia in PROVINCE:
        for anno in ANNO:
            for mese in MESE:
                for tipo_record in TIPO_RECORD_MISE:
                    futures.append(executor.submit(fetch_data, anno, mese, tipo_record, provincia))
    
    for future in concurrent.futures.as_completed(futures):
        future.result()  # Verifica se ci sono eccezioni

# Creazione del DataFrame
columns = ['Anno', 'Mese', 'Provincia', 'DescrizioneProdotto', 'QuotazioneMinima', 'QuotazioneMassima', 'QuotazioneMedia', 'TipologiaProdotto']
df_province = pd.DataFrame(all_data, columns=columns)
df_province

,Anno,Mese,Provincia,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2024,1,Alessandria,Lavatura E Stiratura Gonna (1 Pz),4.5,6.5,5.24,Servizi
1,2024,1,Alessandria,Lavatura E Stiratura Pantalone Uomo (1 Pz),4.5,6.5,5.43,Servizi
2,2024,1,Alessandria,Lavatura E Stiratura Camicia (1 Pz),2.5,6.5,3.91,Servizi
3,2024,1,Alessandria,Riparazione Pantalone (1 Pz),5,10,7.86,Servizi
4,2024,1,Alessandria,Risolatura Scarpe Uomo (1 Pz),18,35,24.47,Servizi
...,...,...,...,...,...,...,...,...
53964,2024,7,Vicenza,Bagno/doccia Schiuma (250 Ml),0.44,2.15,1.11,Cura Della Persona E Della Casa
53965,2024,7,Vicenza,Sapone Liquido (300 Ml),0.33,2,0.67,Cura Della Persona E Della Casa
53966,2024,7,Vicenza,Sapone Toletta (1000 Gr),7.76,14,8.89,Cura Della Persona E Della Casa
53967,2024,7,Vicenza,Dentifricio (100 Ml),1.18,4.41,1.73,Cura Della Persona E Della Casa


In [16]:
# DataFrame ordinato per 'Tipologia di Prodotto', 'Anno', 'Mese', e 'Descrizione Prodotto'
df_province = df_province.sort_values(['Provincia', 'Anno', 'Mese', 'TipologiaProdotto', 'DescrizioneProdotto']).reset_index(drop=True)
df_province

,Anno,Mese,Provincia,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2024,1,Alessandria,Aceto Di Vino (100 Cl),0.65,1.07,0.89,Alimentari
1,2024,1,Alessandria,Acqua Minerale (900 Cl),1.08,4,2.37,Alimentari
2,2024,1,Alessandria,Bastoncini Di Pesce Surgelati (1000 Gr),7.17,10.5,8.91,Alimentari
3,2024,1,Alessandria,Bevanda Gassata (100 Cl),0.57,2.7,1.5,Alimentari
4,2024,1,Alessandria,Bevande Vegetali (1000 Ml),1.43,3.29,2.13,Alimentari
...,...,...,...,...,...,...,...,...
53964,2024,7,Vicenza,Taglio Capelli Uomo (1 Pz),21,33,25.2,Servizi
53965,2024,7,Vicenza,Taxi - Corsa Per L'aeroporto (1 Pz),150,170,159.69,Servizi
53966,2024,7,Vicenza,Toilette Cani (1 Pz),22,50,34.93,Servizi
53967,2024,7,Vicenza,Trasporti Urbani Su Bus - Biglietto (1 Pz),1.7,1.7,1.7,Servizi


In [17]:
df_province.to_csv(f'prezzi_province_{df_province["Anno"].iloc[0]}-{df_province["Mese"].iloc[0]:02d}_{df_province["Anno"].iloc[-1]}-{df_province["Mese"].iloc[-1]:02d}.csv', index=False)
df_province.to_excel(f'prezzi_province_{df_province["Anno"].iloc[0]}-{df_province["Mese"].iloc[0]:02d}_{df_province["Anno"].iloc[-1]}-{df_province["Mese"].iloc[-1]:02d}.xlsx', index=False)

In [19]:
data_province = pd.read_csv("prezzi_province_2024-01_2024-07.csv")
data_province

,Anno,Mese,Provincia,DescrizioneProdotto,QuotazioneMinima,QuotazioneMassima,QuotazioneMedia,TipologiaProdotto
0,2024,1,Alessandria,Aceto Di Vino (100 Cl),0.65,1.07,0.89,Alimentari
1,2024,1,Alessandria,Acqua Minerale (900 Cl),1.08,4.00,2.37,Alimentari
2,2024,1,Alessandria,Bastoncini Di Pesce Surgelati (1000 Gr),7.17,10.50,8.91,Alimentari
3,2024,1,Alessandria,Bevanda Gassata (100 Cl),0.57,2.70,1.50,Alimentari
4,2024,1,Alessandria,Bevande Vegetali (1000 Ml),1.43,3.29,2.13,Alimentari
...,...,...,...,...,...,...,...,...
53964,2024,7,Vicenza,Taglio Capelli Uomo (1 Pz),21.00,33.00,25.20,Servizi
53965,2024,7,Vicenza,Taxi - Corsa Per L'aeroporto (1 Pz),150.00,170.00,159.69,Servizi
53966,2024,7,Vicenza,Toilette Cani (1 Pz),22.00,50.00,34.93,Servizi
53967,2024,7,Vicenza,Trasporti Urbani Su Bus - Biglietto (1 Pz),1.70,1.70,1.70,Servizi


In [20]:
data_province.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53969 entries, 0 to 53968
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Anno                 53969 non-null  int64  
 1   Mese                 53969 non-null  int64  
 2   Provincia            53969 non-null  object 
 3   DescrizioneProdotto  53969 non-null  object 
 4   QuotazioneMinima     53969 non-null  float64
 5   QuotazioneMassima    53969 non-null  float64
 6   QuotazioneMedia      53969 non-null  float64
 7   TipologiaProdotto    53969 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 3.3+ MB
